In [1]:
import optuna
import pickle
import plotly 

In [2]:
study_name = "crps_obj_CRPS_twCRPS12_MOTPE.pkl"
filepath = '/net/pc200239/nobackup/users/hakvoort/hyperopt/hyperopt_emos/'

study = pickle.load(open(filepath + study_name, 'rb'))

In [3]:
best_trials = study.best_trials

for trial in best_trials:
    print(trial.params)
    print(trial.values)
    print()


{'chain_function_mean': 9.740838762394072, 'chain_function_std': 2.2525553280046933, 'chain_function_constant': 0.3614450801580585, 'optimizer': 'Adam', 'learning_rate': 0.06543755067006281, 'forecast_distribution': 'distr_trunc_normal', 'perform_batching': False, 'epochs': 511}
[0.8872740864753723, 0.06216616928577423]

{'chain_function_mean': 8.86543346748238, 'chain_function_std': 0.0153188130495292, 'chain_function_constant': 0.12907501096642482, 'optimizer': 'SGD', 'learning_rate': 0.050750550753121924, 'forecast_distribution': 'distr_trunc_normal', 'perform_batching': True, 'batch_size': 111, 'epochs': 570}
[0.8876388669013977, 0.061799779534339905]

{'chain_function_mean': 8.961787428122632, 'chain_function_std': 1.5232658267824235, 'chain_function_constant': 0.0009608066730378219, 'optimizer': 'SGD', 'learning_rate': 0.07228549130694825, 'forecast_distribution': 'distr_gev', 'perform_batching': True, 'batch_size': 93, 'epochs': 59}
[0.9566889405250549, 0.061194151639938354]

{'

In [5]:
optuna.visualization.plot_pareto_front(study, target_names=['CRPS', 'twCRPS12'])